In [ ]:
%pylab inline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# irisデータセットの分類

In [ ]:
from sklearn.datasets import load_iris
data = load_iris()

In [ ]:
data['data'][48:53]

In [ ]:
data['feature_names']

In [ ]:
data['target'][48:53]

クラス0が `'setosa'`, クラス1が `'versicolor'`, クラス2が `'virginica'` というアヤメの種類

In [ ]:
data['target_names']

In [ ]:
X = torch.tensor(data['data'], dtype=torch.float)
y = torch.tensor(data['target'], dtype=torch.long)

In [ ]:
class MLP(nn.Module):
    def __init__(self, xdim, hdim, ydim):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(xdim, hdim)
        self.linear2 = nn.Linear(hdim, ydim)
        
    def forward(self, X):
        a1 = self.linear1(X)
        z1 = F.relu(a1)
        a2 = self.linear2(z1)
        return a2

In [ ]:
mlp = MLP(4, 2, 3)
optimizer = optim.SGD(mlp.parameters(), lr = 0.1)
for epoch in range(100):
    optimizer.zero_grad()
    a = mlp(X)
    loss = F.cross_entropy(a, y)
    loss.backward()
    optimizer.step()
    print(loss)

In [ ]:
a = mlp(X)
pred_proba = F.softmax(a, dim=1)
pred_y = torch.argmax(pred_proba, dim=1)
print(pred_y)
print(pred_y == y)
accuracy = (pred_y == y).sum() / float(len(pred_y))
print(accuracy)

# 訓練データとテストデータ

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_idx, test_idx = train_test_split(np.arange(len(X)), test_size=0.3, random_state=42)
train_X = torch.tensor(data['data'][train_idx], dtype=torch.float)
train_y = torch.tensor(data['target'][train_idx], dtype=torch.long)
test_X = torch.tensor(data['data'][test_idx], dtype=torch.float)
test_y = torch.tensor(data['target'][test_idx], dtype=torch.long)

In [ ]:
print(len(train_X), len(train_y))
print(len(test_X), len(test_y))

In [ ]:
mlp = MLP(4, 3, 3)
optimizer = optim.SGD(mlp.parameters(), lr = 0.1)
for epoch in range(100):
    optimizer.zero_grad()
    a = mlp(train_X)
    loss = F.cross_entropy(a, train_y)
    loss.backward()
    optimizer.step()
    print(loss)

In [ ]:
a = mlp(test_X)
pred_proba = F.softmax(a, dim=1)
pred_y = torch.argmax(pred_proba, dim=1)
print(pred_y)
print(pred_y == test_y)
accuracy = (pred_y == test_y).sum() / float(len(pred_y))
print(accuracy)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(test_y.data, pred_y.data, target_names=data['target_names']))